In [47]:
import minigrid
from minigrid.wrappers import ImgObsWrapper
from stable_baselines3 import PPO
import torch.nn as nn

import torch
import gymnasium as gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from typing import Dict


In [119]:
class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 512, normalized_image: bool = False) -> None:
        super().__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[-1]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

In [67]:
torch.as_tensor(env.observation_space.sample()[None]).float().shape[1]

7

In [120]:


policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)

env = gym.make("MiniGrid-Empty-16x16-v0", render_mode="rgb_array")
env = ImgObsWrapper(env)

model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1)
model.learn(20)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


AttributeError: 'numpy.ndarray' object has no attribute 'permute'

In [118]:
obs = env.reset()
obs = obs[0]
done = False
feature_extractor  = MinigridFeaturesExtractor(env.observation_space, features_dim=128)

while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info, _ = env.step(action)
    obs_copy = obs
    tensor_obs = torch.tensor(obs).float().permute(2,0,1).unsqueeze(0)
    features = feature_extractor(tensor_obs)
    print("done")


RuntimeError: Given groups=1, weight of size [16, 3, 2, 2], expected input[1, 7, 7, 3] to have 3 channels, but got 7 channels instead

In [103]:
model.policy.features_extractor

MinigridFeaturesExtractor(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (linear): Sequential(
    (0): Linear(in_features=1024, out_features=128, bias=True)
    (1): ReLU()
  )
)

In [104]:
feature_extractor

NameError: name 'feature_extractor' is not defined

In [79]:
tensor_obs.shape

torch.Size([1, 3, 7, 7])

In [96]:
obs = env.reset()
obs[0].shape

(7, 7, 3)

In [90]:
obs[0].unsqueeze(0)

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [100]:
obs_copy.shape

(7, 7, 3)